In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2
import json

In [3]:
# load data from json

with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())

insert_table = Data_to_Otomoto["insert_table"]
car_brand = Data_to_Otomoto["car_brand"]
table_cars = Data_to_Otomoto["table_cars"]
CREATE_TABLE = Data_to_Otomoto["CREATE_TABLE"]

In [4]:
html_data = requests.get("https://www.otomoto.pl/oferta/dacia-duster-prestige-130km-faktura-vat-fabryczna-gwarancja-salon-pl-u-ID6DQcyU.html#7ce9f71ee3")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [5]:
eur = 4.57
print(eur)


4.57


In [6]:
def get_price(soup):
    Cena = soup.find('span',attrs={"class":"offer-price__number"}).text
    Cena = Cena.replace(" ","").replace(",",".")
    if "PLN" in Cena:
        Cena = Cena.replace("PLN","")
        Cena = float(Cena)
    elif "EUR" in Cena:
        Cena = Cena.replace("EUR","")
        Cena = float(Cena)
        Cena = Cena * eur
    else:
        Cena = np.NaN
    return Cena

In [7]:
print(get_price(soup))

69900.0


In [8]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [9]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip().replace(" ","_").replace("(","").replace(")","")
        key = key.replace("ś","s").replace("ż","z").replace("ó","o").replace("ć","c").replace("ę","e")
        key = key.replace("ł","l").replace("ą","a")
        if key in insert_table:
            value = info.find('div').text.strip()
            details[key] = value
    return details

In [10]:
def take_all(soup):
    try:
        dict_all = {}
        dict_all["ID"] = take_id(soup)
        dict_all["Cena"] = get_price(soup)
        details = take_details(soup)
        return dict_all | details
    except:
        pass

In [11]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'].replace("km","").replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'].replace("KM","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Pojemnosc_skokowa'] = details['Pojemnosc_skokowa'].replace("cm3","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Miescie'] = details['Spalanie_W_Miescie'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Cyklu_Mieszanym'] = details['Spalanie_W_Cyklu_Mieszanym'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_Poza_Miastem'] = details['Spalanie_Poza_Miastem'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Emisja_CO2'] = details['Emisja_CO2'].replace("g/km","").replace(" ","").replace(",",".")
    except:
        pass
    return details

In [12]:
car_brand = ["abarth"]
#         "alfa-romeo",
#         "aston-martin",
#         "audi",
#         "bentley",
#         "bmw",
#         "bugatti",
#         "cadillac",
#         "chevrolet",
#         "chrysler",
#         "citroën",
#         "dacia",
#         "daewoo",
#         "daihatsu",
#         "dodge",
#         "donkervoort",
#         "ds",
#         "ferrari",
#         "fiat",
#         "fisker",
#         "ford",
#         "honda",
#         "hummer",
#         "hyundai",
#         "infiniti",
#         "iveco",
#         "jaguar",
#         "jeep",
#         "kia",
#         "ktm",
#         "lada",
#         "lamborghini",
#         "lancia",
#         "land-rover",
#         "landwind",
#         "lexus",
#         "lotus",
#         "maserati",
#         "maybach",
#         "mazda",
#         "mclaren",
#         "mercedes-benz",
#         "mg",
#         "mini",
#         "mitsubishi",
#         "morgan",
#         "nissan",
#         "opel",
#         "peugeot",
#         "porsche",
#         "renault",
#         "rolls-royce",
#         "rover",
#         "saab",
#         "seat",
#         "skoda",
#         "smart",
#         "ssangYong",
#         "subaru",
#         "suzuki",
#         "tesla",
#         "toyota",
#         "volkswagen",
#         "volvo"
            

In [13]:
table_cars = pd.DataFrame(table_cars)

In [14]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,10): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            if brand in html_data_link.url:       
                soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
                links = []
                for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
                    if "https://www.otomoto.pl" in offer['href']:
                        links.append(offer['href']) 
                for page in links:
                    print(page)
                    html_data = requests.get(page)
                    soup = BeautifulSoup(html_data.content, 'html5lib')
                    dict_all = take_all(soup)
                    dict_all = data_clean(dict_all)
                    dict_all["Link"] = page
                    if dict_all:
                        all_data.append(dict_all)
                    Read_announcements += 1
                    clear_output(wait=True)
                    print("Read_announcements ", Read_announcements) 
        except Exception:
            print("Link_ERROR")
            continue
table_cars = table_cars.append(all_data, ignore_index=True)
#table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  61
Delete duplicates:  0
Finish


In [15]:
# page_link = brand_link + str(page_main)
# print(page_link)

In [16]:
res=requests.get('https://www.otomoto.pl/osobowe/abarth/?search%5Border%5D=created_at%3Adesc&page=10')
res.url

'https://www.otomoto.pl/osobowe/?page=2'

In [17]:
links

['https://www.otomoto.pl/oferta/abarth-grande-punto-fiat-punto-abarth-grande-punto-super-stan-polecam-ID6DMdbX.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-lancia-abart-a112-ID6DNBYH.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-595-abarth-scorpioneoro-165-km-edycja-limitowana-2000-szt-ID6DEoz4.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-595-1-4-16v-t-jet-165km-scorpioneoro-2021-salon-r-schewe-ID6DFq0t.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-595-595c-1-4-16v-t-jet-165-km-turismo-tel-6-6-7-8-8-0-7-8-0-bartek-2021-ID6E17Wc.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-grande-punto-pl-krajowy-maly-przebieg-radio-android-2-koplety-kol-ID6DOfXi.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-595-anniversario-1-4t-jet-180km-nawigacja-system-audio-hi-fi-beats-opcje-ID6E36iG.html#f133bffbe6',
 'https://www.otomoto.pl/oferta/abarth-595-1-4-595-serie-5-t-jet-145km-super-cena-ID6E36io.html#f133bffbe6',
 'https://www.otomoto.

In [18]:
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# table_cars.head()

In [19]:
# #GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,4):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

In [20]:
table_cars.head()

,ID,Cena,Oferta_od,Kategoria,Marka_pojazdu,Model_pojazdu,Rodzaj_paliwa,Moc,Pojemnosc_skokowa,Liczba_drzwi,...,Miesieczna_rata,lub_do_przebieg_km,Gwarancja_dealerska_w_cenie,Homologacja_ciezarowa,Oplata_poczatkowa,Liczba_pozostalych_rat,Wartosc_wykupu,Kierownica_po_prawej_Anglik,Zarejestrowany_jako_zabytek,Link
0,6088515478,61900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.otomoto.pl/oferta/abarth-595-darmo...
1,6088033302,71900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.otomoto.pl/oferta/abarth-595-darmo...
2,6088506602,67900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.otomoto.pl/oferta/abarth-595-darmo...
3,6088316230,52800.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.otomoto.pl/oferta/abarth-595-abart...
4,6088233917,82350.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.otomoto.pl/oferta/abarth-595-1-4-1...


In [21]:
table_name = 'Otomoto'

In [22]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

In [23]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [24]:
cur = psql.cursor()

In [25]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + CREATE_TABLE 
# '''(ID float NOT NULL,
#     Cena float,
#     Oferta_od VARCHAR,
#     Kategoria VARCHAR,
#     Marka_pojazdu VARCHAR,
#     Model_pojazdu VARCHAR,
#     Rodzaj_paliwa VARCHAR,
#     Moc float,
#     Pojemnosc_skokowa float,
#     Liczba_drzwi float,
#     Skrzynia_biegow VARCHAR,
#     Wersja VARCHAR,
#     Generacja VARCHAR,
#     Segment VARCHAR,
#     Kolor VARCHAR,
#     Przebieg float,
#     Rok_produkcji float,
#     Naped VARCHAR,
#     Spalanie_W_Miescie float,
#     Typ_nadwozia VARCHAR,
#     Emisja_CO2 float,
#     Liczba_miejsc float,
#     Rodzaj_koloru VARCHAR,
#     Kraj_pochodzenia VARCHAR,
#     Pierwszy_wlasciciel VARCHAR,
#     Bezwypadkowy VARCHAR,
#     Serwisowany_w_ASO VARCHAR,
#     Stan VARCHAR,
#     VAT_marza VARCHAR,
#     Pierwsza_rejestracja VARCHAR,
#     Numer_rejestracyjny_pojazdu VARCHAR,
#     Zarejestrowany_w_Polsce VARCHAR,
#     Filtr_czastek_stalych VARCHAR,
#     Tuning VARCHAR,
#     Mozliwosc_finansowania VARCHAR,
#     Uszkodzony VARCHAR,
#     Faktura_VAT VARCHAR,
#     Spalanie_W_Cyklu_Mieszanym float,
#     Spalanie_Poza_Miastem float,
#     Okres_gwarancji_producenta VARCHAR,
#     Leasing VARCHAR,
#     Miesieczna_rata VARCHAR,
#     lub_do_przebieg_km VARCHAR,
#     Gwarancja_dealerska_w_cenie VARCHAR,
#     Homologacja_ciezarowa VARCHAR,
#     Oplata_poczatkowa VARCHAR,
#     Liczba_pozostalych_rat VARCHAR,
#     Wartosc_wykupu VARCHAR,
#     Kierownica_po_prawej_Anglik VARCHAR,
#     Zarejestrowany_jako_zabytek VARCHAR,
#     PRIMARY KEY (ID)
#     )'''
cur.execute(createTableCommand)
psql.commit()

In [26]:
a = 0
for row in table_cars.itertuples(index=False):
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+') ON CONFLICT DO NOTHING;'
    insertDataCommand = full_fit+"\r\n"+Value_data
    
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

Finish
